In [2]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
import os

In [3]:
os.environ["OPENAI_API_KEY"]='xxxxxx'
os.environ["SERPAPI_API_KEY"] = "xxxxx"
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [4]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"Meet Leonardo DiCaprio's new model girlfriend. Leonardo DiCaprio has moved on from his months-long fling with Gigi Hadid with Italian model Vittoria Ceretti. I need to find out Vittoria Ceretti's age.
Action: Search
Action Input: "Vittoria Ceretti age"['Vittoria Ceretti is an Italian model. Ceretti was discovered in 2012 through the Elite Model Look Model contest. Models.com ranks Ceretti as one of the "New Supers" of this generation.', 'Vittoria Ceretti type: Italian model.', 'Vittoria Ceretti born: June 7, 1998 (age 25 years), Brescia, Italy.', 'Vittoria Ceretti spouse: Matteo Milleri (m. 2020–2023).', 'Vittoria Ceretti height: 5′ 9″.', 'Vittoria Ceretti parents: Francesca Lazzari, Giuseppe Ceretti.', 'Vittoria Ceretti siblings: Guglielmo Ceretti.', 'Vittoria Ceretti eye_color: Green.', 

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-rVVCSroW43WKcfaVt5n3w4kN on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [11]:
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Vittoria Ceretti
Thought: I need to find out Vittoria Ceretti's age
Action: Search
Action Input: "Vittoria Ceretti age"
Observation: 25 years
Thought: I need to calculate 25 raised to the 0.43 power
Action: Calculator
Action Input: 25^0.43

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-rVVCSroW43WKcfaVt5n3w4kN on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}